### tf.name_scope()

In [26]:
from keras import backend as K
K.clear_session() 

import tensorflow as tf

with tf.name_scope('a_name_scope'):
    
    initializer = tf.constant_initializer(value=1)
    var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32, initializer=initializer)   # 创建变量方式1
    
    var2 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)                        # 创建变量方式2
    var21 = tf.Variable(name='var2', initial_value=[2.1], dtype=tf.float32)
    var22 = tf.Variable(name='var2', initial_value=[2.2], dtype=tf.float32)
    
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))
    print(var21.name, sess.run(var21))
    print(var22.name, sess.run(var22))

var1:0 [1.]
a_name_scope/var2:0 [2.]
a_name_scope/var2_1:0 [2.1]
a_name_scope/var2_2:0 [2.2]


### tf.variable_scope()

In [27]:
from keras import backend as K
K.clear_session() 

with tf.variable_scope('a_variable_scope') as scope:
    
    initializer = tf.constant_initializer(value=3)
    var3 = tf.get_variable(name='var3', shape=[1], dtype=tf.float32, initializer=initializer)   # 采用 tf.variable_scope 时，变量名前有了‘a_variable_scope/’
    
    var4 = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)                        
    var4_reuse = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)                  # 不能重复调用
    
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(var3.name, sess.run(var3))
    print(var4.name, sess.run(var4))
    print(var4_reuse.name, sess.run(var4_reuse))

a_variable_scope/var3:0 [3.]
a_variable_scope/var4:0 [4.]
a_variable_scope/var4_1:0 [4.]


### 重复调用变量

In [28]:
from keras import backend as K
K.clear_session() 

with tf.variable_scope('a_variable_scope') as scope:    # 调用 tf.variable_scope()
    
    initializer = tf.constant_initializer(value=3)
    var3 = tf.get_variable(name='var3', shape=[1], dtype=tf.float32, initializer=initializer) 
    
    scope.reuse_variables()                        # 声明其后变量重复调用
    var3_reuse = tf.get_variable(name='var3')      # 重复调用
    
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(var3.name, sess.run(var3))
    print(var3_reuse.name, sess.run(var3_reuse))

a_variable_scope/var3:0 [3.]
a_variable_scope/var3:0 [3.]


### RNN示例

In [78]:
import tensorflow as tf

from keras import backend as K
K.clear_session()

class TrainConfig:
    batch_size = 20
    time_steps = 20
    input_size = 10
    output_size = 2
    cell_size = 11
    learning_rate = 0.01
    
class TestConfig(TrainConfig):
    time_steps = 1               # Test 中改变 time_step
    
class RNN:
    def __init__(self, config):
        self._batch_size = config.batch_size
        self._time_steps = config.time_steps
        self._input_size = config.input_size
        self._output_size = config.output_size
        self._cell_size = config.cell_size
        self._lr = config.learning_rate
        self._built_RNN()
        
    def _built_RNN(self):
        with tf.variable_scope('inputs'):
            self._xs = tf.placeholder(tf.float32, [self._batch_size, self._time_steps, self._input_size], name='xs')
            self._ys = tf.placeholder(tf.float32, [self._batch_size, self._time_steps, self._output_size], name='ys')
        with tf.name_scope('RNN'):
            with tf.variable_scope('input_layer'):
                l_in_x = tf.reshape(self._xs, [-1, self._input_size], name='2_2D')   # (batch*n_step, in_size)
                Wi = self._weight_variable([self._input_size, self._cell_size])      # Ws(in_size, cell_size)
                print(Wi.name)
                bi = self._bias_variable([self._cell_size,])                         # bs(cell_size, )
                # l_in_y(batch*n_steps, cell_size)
                with tf.name_scope('Wx_plus_b'):
                    l_in_y = tf.matmul(l_in_x, Wi)+bi
                l_in_y = tf.reshape(l_in_y, [-1, self._time_steps, self._cell_size], name='2_3D')                  
                
            with tf.variable_scope('lstm_cell'):
                # cell = tf.nn.rnn_cell.BasicRNNCell(self._cell_size)
                cell = tf.contrib.rnn.BasicLSTMCell(self._cell_size)
                with tf.name_scope('initial_state'):
                    self._cell_initial_state = cell.zero_state(self._batch_size, dtype=tf.float32)
        
                self.cell_outputs = []
                cell_state = self._cell_initial_state
                for t in range(self._time_steps):
                    if t > 0:
                        tf.get_variable_scope().reuse_variables()
                    cell_output, cell_state = cell(l_in_y[:, t, :], cell_state)
                    self.cell_outputs.append(cell_output)
                self._cell_final_state = cell_state
            
            with tf.variable_scope('output_layer'):
                # cell_outputs_reshaped(BATCH*TIME_STEP, CELL_SIZE)
                cell_outputs_reshaped = tf.reshape(tf.concat(self.cell_outputs, 1), [-1, self._cell_size])
                Wo = self._weight_variable((self._cell_size, self._output_size))
                bo = self._bias_variable((self._output_size,))
                product = tf.matmul(cell_outputs_reshaped, Wo) + bo
                # _pred shape(batch*time_step, output_size)
                self._pred = tf.nn.relu(product)            # for displacement
                
        with tf.name_scope('cost'):
            _pred = tf.reshape(self._pred, [self._batch_size, self._time_steps, self._output_size])
            mse = self.ms_error(_pred, self._ys)
            mse_ave_across_batch = tf.reduce_mean(mse, 0)
            mse_sum_across_time = tf.reduce_sum(mse_ave_across_batch, 0)
            self._cost = mse_sum_across_time
            self._cost_ave_time = self._cost/self._time_steps
                
        with tf.variable_scope('train'):
            self._lr = tf.convert_to_tensor(self._lr)
            self.train_op = tf.train.AdamOptimizer(self._lr).minimize(self._cost)
                
    @staticmethod
    def ms_error(y_target, y_pre):
        return tf.square(tf.subtract(y_target, y_pre))
        
    @staticmethod
    def _weight_variable(shape, name='weights'):
        initizlizer = tf.random_normal_initializer(mean=0., stddev=0.5,)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)
        
    @staticmethod
    def _bias_variable(shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(shape=shape, initializer=initializer, name=name)
        
if __name__ == '__main__':
    train_config = TrainConfig()
    test_config = TestConfig()
    
    # 不采用方法，直接调用（报错）
    # train_rnn1 = RNN(train_config)       # 第一次调用 weight，输出 input_layer/weights:0
    # test_rnn1 = RNN(test_config)         # 第二次调用 weight，报错 ValueError: Variable input_layer/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:
    
    # 采用 tf.variable_scope() 解决重复调用问题（并未达到共享变量的作用，两次调用名称不同，为不同变量）
    # with tf.variable_scope('train_rnn'):  
    #     train_rnn1 = RNN(train_config)    # 第一次调用 weight，输出 train_rnn/input_layer/weights:0
    # with tf.variable_scope('test_rnn'):
    #     test_rnn1 = RNN(test_config)      # 第二次调用 weight，输出 test_rnn/input_layer/weights:0
        
    # 采用 tf.variable_scope() 和 scope.reuse_variables() 解决共享变量问题
    with tf.variable_scope('rnn') as scope:
        sess = tf.Session()
        train_rnn2 = RNN(train_config)     # 训练参数                   输出 rnn/input_layer/weights:0
        
        scope.reuse_variables()            # 共享变量申明
        test_rnn2 = RNN(test_config)       # 重复利用 train 中的参数    输出 rnn/input_layer/weights:0
        sess.run(tf.initialize_all_variables())

rnn/input_layer/weights:0
rnn/input_layer/weights:0


### tf.square()，对各元素求平方

In [81]:
import tensorflow as tf
test = tf.Graph()
with test.as_default():
    X = tf.constant([1, 2, 3, 4], dtype=tf.float32, name=None)
    Y = tf.square(X)         # 平方操作
    
    with tf.Session() as sess:
        print(sess.run(Y))

[ 1.  4.  9. 16.]


### tf.reduce_mean(input_tensor, axis=None, keep_dims=False, name=None)，指定维度求均值，默认求所有元素均值
* input_tesor，输入张量
* axis，指定轴
* keep_dims，是否降维。设置 True，结果保持输入 Tensor 的形状（维度）；设置 False，为降维后的结果；默认 False
* name, 名称

In [88]:
import tensorflow as tf

test = tf.Graph()
with test.as_default():
    X = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32, name=None)
    Y = tf.reduce_mean(X)                # 求所有元素均值
    Y0 = tf.reduce_mean(X, axis=0)       # 求 0 维度均值，即列均值
    Y1 = tf.reduce_mean(X, axis=1)       # 求 1 维度均值，即行均值
    
    Y0_ = tf.reduce_mean(X, axis=0, keep_dims=True)       # 求行均值，保持输入形状
    Y1_ = tf.reduce_mean(X, axis=1, keep_dims=True)       # 求行均值，保持输入形状
    
    with tf.Session() as sess:
        print(sess.run(Y))
        print(sess.run(Y0))
        print(sess.run(Y1))
        print(sess.run(Y0_))
        print(sess.run(Y1_))

3.5
[2.5 3.5 4.5]
[2. 5.]
[[2.5 3.5 4.5]]
[[2.]
 [5.]]


### 类似函数：
* tf.reduce_sum ：计算tensor指定轴方向上的所有元素的累加和；
* tf.reduce_max  :  计算tensor指定轴方向上的各个元素的最大值；
* tf.reduce_all :  计算tensor指定轴方向上的各个元素的逻辑和（and运算）；
* tf.reduce_any:  计算tensor指定轴方向上的各个元素的逻辑或（or运算）
    
**参考**：[csdn博客](https://blog.csdn.net/dcrmg/article/details/79797826)

### tf.reshape(tensor, shape, name=None)
* 改变 tensor 的 shape
* shape 设置为 -1 的位置，表示不用设置这一维的大小，函数自动进行计算（只能存在一个 -1）

In [89]:
import tensorflow as tf

test = tf.Graph()
with test.as_default():
    X = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32, name=None)
    Y1 = tf.reshape(X, (3, 2))        # (2, 3) --> (3, 2)
    Y2 = tf.reshape(X, (-1, 6))       # (2, 3) --> (1, 6)
    Y3 = tf.reshape(X, (6, -1))       # (2, 3) --> (6, 1)
    
    with tf.Session() as sess:
        print('原Tensor:\n', sess.run(X))
        print('(3, 2):\n', sess.run(Y1))
        print('(1, 6):\n', sess.run(Y2))
        print('(6, 1):\n', sess.run(Y3))

原Tensor:
 [[1. 2. 3.]
 [4. 5. 6.]]
(3, 2):
 [[1. 2.]
 [3. 4.]
 [5. 6.]]
(1, 6):
 [[1. 2. 3. 4. 5. 6.]]
(6, 1):
 [[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]]


### tf.concat([tensor1, tensor2, tensor3, ... ], axis)，拼接张量
* axis，指定轴（维度）

In [91]:
import tensorflow as tf

test = tf.Graph()
with test.as_default():
    t1 = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
    t2 = tf.constant([[7, 8, 9], [10, 11, 12]], dtype=tf.float32)
    T0 = tf.concat([t1, t2], axis=0)      # 维度 0 处拼接，即列拼接
    T1 = tf.concat([t1, t2], axis=1)      # 维度 1 处拼接，即行拼接
    with tf.Session() as sess:
        print(sess.run(T0))
        print(sess.run(T1))

[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]
[[ 1.  2.  3.  7.  8.  9.]
 [ 4.  5.  6. 10. 11. 12.]]
